In [ ]:
import pandas as pd

# File paths
fasta_file = 'fasta_files.csv'
gff_file = 'gff_files.csv'
prefix_file = 'gff-assembly-prefixes.tsv'
species_file = 'species.csv'
strain_file = 'strain.csv'

# Load the CSV files into dataframes
fasta_df = pd.read_csv(fasta_file)
gff_df = pd.read_csv(gff_file)
prefix_df = pd.read_csv(prefix_file, sep='\t')
species_df = pd.read_csv(species_file)

# Ensure the correct columns are used for merging
# Extract the isolate name from the fasta file
fasta_df['Isolate Name'] = fasta_df['Assembly Name'].apply(lambda x: x.split('_NT')[0])

# Prepare prefix_df by removing extensions from assembly file names
prefix_df['Assembly'] = prefix_df['assembly'].str.replace('.fa', '')

# Merge fasta_df with prefix_df on Assembly Name to get the GFF Prefix
fasta_merged_df = pd.merge(fasta_df, prefix_df, how='left', left_on='Assembly Name', right_on='Assembly')

# Merge the result with the gff_df on the GFF Prefix (Isolate Name in gff_df)
final_merged_df = pd.merge(fasta_merged_df, gff_df, how='left', left_on='prefix', right_on='Isolate Name')

# Map species IDs based on the species name from species.csv
species_mapping = dict(zip(species_df['Species Name'], species_df['ID']))

# Add species ID column based on Isolate Name prefix and species mapping
final_merged_df['Species ID'] = final_merged_df['Isolate Name_y'].apply(
    lambda x: species_mapping.get('Bacteroides uniformis') if x.startswith('BU_')
    else species_mapping.get('Phocaeicola vulgatus') if x.startswith('PV_')
    else 'Unknown'
)

# Ensure the Species ID is valid
final_merged_df = final_merged_df[final_merged_df['Species ID'] != 'Unknown']

# Select and rename columns for the final output, now including Species ID and other Strain fields
final_df = final_merged_df[['Species ID', 'Isolate Name_y', 'Assembly Name', 'FTP File_x', 'FTP File_y']]
final_df.columns = ['Species ID', 'Isolate Name', 'Assembly Name', 'Fasta File', 'GFF File']

# Add strain name column after the isolate name
final_df.insert(2, 'Strain Name', final_df['Isolate Name'])  # Assuming Strain Name is the same as Isolate Name for now

# Add Assembly Accession as "123456" for now
final_df['Assembly Accession'] = '123456'

# Save the merged dataframe to a new CSV file
final_df.to_csv(strain_file, index=False)

print(f"Strain file '{strain_file}' has been created with the merged records.")
